In [2]:
import requests as r
import pandas as pd
from bs4 import BeautifulSoup

In [41]:
BASE_URL = "https://blokkenschema.nl"

In [14]:
res = r.get(BASE_URL+ '/lowlands-2023/line-up')

In [16]:
soup = BeautifulSoup(res.text, 'html.parser')
hl_links = [li.a['href'] for li in soup.find_all('li', class_='headliner')]

In [92]:
def get_schema_data(link):
    res_hl  = r.get(BASE_URL + link)
    soup_hl = BeautifulSoup(res_hl.text, 'html.parser')
    hl_data = soup_hl.find('div', class_='performance-data')
    return {
        'name': soup_hl.find('div', itemprop='performer').find('h3').string,
        'location': hl_data.find('div', itemprop='location').string,
        'start_date': hl_data.find('div', itemprop='startDate')['content'],
        'end_date': hl_data.find('span', itemprop='endDate')['content'],
    }

In [94]:
list_of_schema = [get_schema_data(link) for link in hl_links]
df = pd.DataFrame(list_of_schema)
df.to_csv("lowlands_schema.csv")

In [3]:
## write dataframe to a database with sqlalchemy engine
from sqlalchemy import create_engine
from config import Config

df = pd.read_csv('lowlands_schema.csv')

df['startdate'] = df['startdate'].str.replace("+02:00", "")
df['enddate'] = df['enddate'].str.replace("+02:00", "")

df['startdate'] = pd.to_datetime(df['startdate'])
df['enddate'] = pd.to_datetime(df['enddate'])

df.head()

,id,name,stage,startdate,enddate
0,0,(Future) Festival Classics,Heineken,2023-08-18 23:00:00,2023-08-19 01:00:00
1,1,¥///0 $#£[[,India,2023-08-18 21:55:00,2023-08-18 22:55:00
2,2,070 Shake,India,2023-08-19 19:00:00,2023-08-19 20:00:00
3,3,155 & Maas Theater En Dans: Controle,Juliet,2023-08-19 17:00:00,2023-08-19 18:00:00
4,4,Abel Van Gijlswijk: Aabz ’23 Tour – Rebranding...,Juliet,2023-08-18 17:00:00,2023-08-18 18:00:00


In [4]:
df.columns

Index(['id', 'name', 'stage', 'startdate', 'enddate'], dtype='object')

In [5]:
df['festival'] = "Lowlands"

In [6]:
df.head()

,artist_id,name,stage,startdate,enddate,festival
0,0,(Future) Festival Classics,Heineken,2023-08-18 23:00:00,2023-08-19 01:00:00,Lowlands
1,1,¥///0 $#£[[,India,2023-08-18 21:55:00,2023-08-18 22:55:00,Lowlands
2,2,070 Shake,India,2023-08-19 19:00:00,2023-08-19 20:00:00,Lowlands
3,3,155 & Maas Theater En Dans: Controle,Juliet,2023-08-19 17:00:00,2023-08-19 18:00:00,Lowlands
4,4,Abel Van Gijlswijk: Aabz ’23 Tour – Rebranding...,Juliet,2023-08-18 17:00:00,2023-08-18 18:00:00,Lowlands


In [7]:
cfg = Config()
engine = create_engine(cfg.SQLALCHEMY_DATABASE_URI, echo=True)
df.to_sql('artist', con=engine, if_exists='append', index=False)

238